In [ ]:
import json
import time

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = REPLACEME
credentials['CONSUMER_SECRET'] = REPLACEME
credentials['ACCESS_TOKEN'] = REPLACEME
credentials['ACCESS_SECRET'] = REPLACEME

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

In [ ]:
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [ ]:
spotify_urls = []
MAX_ATTEMPTS = 2000
COUNT_OF_URLS_TO_BE_FETCHED = 10000

In [ ]:
for i in range(0,MAX_ATTEMPTS):

    if(COUNT_OF_URLS_TO_BE_FETCHED < len(spotify_urls)):
        break # we got 5000 urls... !!

    #----------------------------------------------------------------#
    # STEP 1: Query Twitter
    # STEP 2: Save the returned tweets
    # STEP 3: Get the next max_id
    #----------------------------------------------------------------#

    # STEP 1: Query Twitter
    if(0 == i):
        # Query twitter for data. 
        results = python_tweets.search(q="url:open.spotify.com/playlist", include_entities='true', count='100')
    else:
        # After the first call we should have max_id from result of previous call. Pass it in query.
        results = python_tweets.search(q="url:open.spotify.com/playlist", include_entities='true',max_id=next_max_id)

    # STEP 2: Save the returned tweets
    for result in results['statuses']:
        entities = result['entities']
        for entity in entities:
            if entity == 'urls':
                urls = entities['urls']
                for url in urls:
                    if(url['expanded_url'] and 'open.spotify.com/playlist' in url['expanded_url']):
                        playlist_url = url['expanded_url']
                        spotify_urls.append(playlist_url)
                        

    # STEP 3: Get the next max_id
    try:
        # Parse the data returned to get max_id to be passed in consequent call.
        next_results_url_params = results['search_metadata']['next_results']
        next_max_id = next_results_url_params.split('max_id=')[1].split('&')[0]
        rate_limit_remaining = python_tweets.get_lastfunction_header('x-rate-limit-remaining')
        print(rate_limit_remaining)
        if int(rate_limit_remaining) < 25:
            rate_limit_remaining = python_tweets.get_lastfunction_header('x-rate-limit-remaining')
            print("Next Max Id: {}".format(next_max_id))
            print("Rate limit remaining: {}".format(rate_limit_remaining))
            print("Number of urls scraped: {}".format(len(spotify_urls)))
        time.sleep(2.2)
    except Exception as e:
        # No more next pages
        print(e)
        print('no more')
        break

In [ ]:
len(spotify_urls)

In [ ]:
final_urls = set(spotify_urls)
with open('playlist.csv', 'w') as f:
    for url in final_urls:
        f.write("%s\n" % url)